# imports:

In [1]:
from sklearn.svm import SVR as svr
from sklearn.svm import NuSVR
from sklearn.ensemble import ExtraTreesRegressor as etr
from sklearn.ensemble import RandomForestRegressor as rfr
from sklearn.neighbors import KNeighborsRegressor as Kneigh
from sklearn.ensemble import AdaBoostRegressor as Ada
from sklearn.ensemble import HistGradientBoostingRegressor as HG
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
import uproot
import pickle
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load events

In [2]:
events_P = pd.read_pickle("run3_1to11.pkl")


# remove data where Etrue == 0

# why: tgtvar = eraw/etrue = eraw/0 = inf: therefore models can't be trained

In [3]:
a = 0
len_e = range(len(events_P))
for i in len_e:
    #print(i)
    if (events_P['true'].iloc[i] == 0):
        #print(events_P['true'].iloc[i])
        events_P = events_P.drop(events_P.index[i])
        a += 1
    if a + i == len(len_e) - 1: 
        break
        
        

In [4]:
a = 0
len_e = range(len(events_P))
for i in len_e:
    #print(i)
    if (events_P['true'].iloc[i] == 0):
        #print(events_P['true'].iloc[i])
        events_P = events_P.drop(events_P.index[i])
        a += 1
    if a + i == len(len_e) - 1: 
        break

In [5]:
events_P = events_P.set_axis(range(len(events_P)), axis='index') #set index's again

# sample data

In [6]:
events_P = events_P.sample(90000) 

# get regression input variables and target variable:

In [7]:
eraw =  events_P['ecal']+ events_P['hcal']

In [8]:
ecalDivRaw = events_P['ecal']/eraw

In [20]:
regVars = events_P[['eta','phi','p']]
regVars['ecalDivRaw'] = ecalDivRaw
tgtVar = eraw/events_P['true']
#np.array((eraw - events_P['true'])/events_P['true'])

C:\Users\jonla\AppData\Local\Temp\ipykernel_21540\2027473175.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  regVars['ecalDivRaw'] = ecalDivRaw


# split data

In [21]:
xtrain, xtest, ytrain, ytest, events_train, events_test = train_test_split(regVars, tgtVar,events_P, test_size=0.15)
ytrain = np.ravel(ytrain)

# run models:

In [22]:
#xgbr
XGBR = XGBRegressor(verbosity = 0).fit(xtrain, ytrain)

#ETR
ETR = etr(n_estimators=100, random_state=0).fit(xtrain, ytrain) 

#nusvr
#NSVR_ZS = NuSVR(C=1.0, cache_size=200, coef0=0.0, degree=3, gamma='scale', kernel='rbf',
#      max_iter=-1, nu=0.5, shrinking=True, tol=0.001, verbose=False).fit(xtrain_ZS, ytrain_ZS)

#svr
#SVR_ZS = svr(C=1.0, epsilon=0.2).fit(xtrain_ZS, ytrain_ZS)


#HistGBR
HGBR = HG().fit(xtrain, ytrain)

#ADA
ADA = Ada(random_state=0, n_estimators=100).fit(xtrain, ytrain)

#KNR
KNR = Kneigh(n_neighbors=2).fit(xtrain, ytrain)

#RFR
RFR = rfr(max_depth=2, random_state=0).fit(xtrain, ytrain)

# scores of models:

In [23]:
print(ETR.score(xtest,ytest))
print(XGBR.score(xtest,ytest))

0.4358014279960394
0.4786469059211296


# save models:

In [24]:
pickle.dump(XGBR, open('XGBR_model_run3_1to11.sav', 'wb'))
pickle.dump(ETR, open('ETR_model_run3_1to11.sav', 'wb'))
#pickle.dump(NSVR_ZS, open('NSVR_model_ZS.sav', 'wb'))
#pickle.dump(SVR_ZS, open('SVR_model_ZS.sav', 'wb'))
pickle.dump(HGBR, open('HGBR_model_run3_1to11.sav', 'wb'))
pickle.dump(ADA, open('ADA_model_run3_1to11.sav', 'wb'))
pickle.dump(KNR, open('KNR_model_run3_1to11.sav', 'wb'))
pickle.dump(RFR, open('RFR_model_run3_1to11.sav', 'wb'))

In [25]:
xtest.to_pickle("xtest_run3_1to11.pkl")
ytest.to_pickle("ytest_run3_1to11.pkl")
events_test.to_pickle("events_test_run3_1to11.pkl")